In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,OrdinalEncoder
from sklearn.model_selection import train_test_split

In [3]:
df=pd.read_csv('Titanic-Dataset.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [4]:
df.select_dtypes(include='object').columns

Index(['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'], dtype='object')

In [5]:
le=LabelEncoder()
df['Sex_Encoded']=le.fit_transform(df['Sex'])
df[['Sex','Sex_Encoded']].head()

,Sex,Sex_Encoded
0,male,1
1,female,0
2,female,0
3,female,0
4,male,1


In [6]:
ordinal = OrdinalEncoder(categories=[[1,2,3]])
df["Pclass_encoded"] = ordinal.fit_transform(df[["Pclass"]])
df[["Pclass", "Pclass_encoded"]].head()


,Pclass,Pclass_encoded
0,3,2.0
1,1,0.0
2,3,2.0
3,1,0.0
4,3,2.0


In [8]:
ohe = OneHotEncoder(drop="first", sparse_output=False)


encoded = ohe.fit_transform(df[["Embarked"]])

encoded_df = pd.DataFrame(encoded, columns=ohe.get_feature_names_out(["Embarked"]))
encoded_df.head()


,Embarked_Q,Embarked_S,Embarked_nan
0,0.0,1.0,0.0
1,0.0,0.0,0.0
2,0.0,1.0,0.0
3,0.0,1.0,0.0
4,0.0,1.0,0.0


In [9]:
df = pd.concat([df, encoded_df], axis=1)


In [11]:
df1=pd.read_csv('titanic_cleaned.csv')

In [12]:
target_mean = df1.groupby("Title")["Survived"].mean()
df1["Title_target_encoded"] = df1["Title"].map(target_mean)
df1[["Title","Title_target_encoded"]].head()


,Title,Title_target_encoded
0,Mr,0.156673
1,Mrs,0.792000
2,Miss,0.697802
3,Mrs,0.792000
4,Mr,0.156673


In [14]:
freq = df1["Title"].value_counts()
df1["Title_freq_encoded"] = df1["Title"].map(freq)
df1[["Title","Title_freq_encoded"]].head()


,Title,Title_freq_encoded
0,Mr,517
1,Mrs,125
2,Miss,182
3,Mrs,125
4,Mr,517


In [15]:
freq = df1["Title"].value_counts()
rare_titles = freq[freq < 10].index

df1["Title_grouped"] = df1["Title"].apply(lambda x: "Other" if x in rare_titles else x)
df1["Title_grouped"].value_counts()


,count
Title_grouped,
Mr,517
Miss,182
Mrs,125
Master,40
Other,27


In [19]:
X = df.drop("Survived", axis=1)
y = df["Survived"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

ohe = OneHotEncoder(drop="first", sparse_output=False, handle_unknown="ignore")

ohe.fit(X_train[["Embarked"]])      # fit ONLY on train

train_ohe = ohe.transform(X_train[["Embarked"]])
test_ohe  = ohe.transform(X_test[["Embarked"]])
